Let's do a quick example comparing `jitr` to the standard Runge-Kutta ODE solver implemented in `scipy.integrate.solve_ivp`:

In [23]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import njit
from jitr import reactions, rmatrix, utils
from jitr.reactions.potentials import woods_saxon_potential

First let's check our library versions and configs, as this will affect performance:

In [24]:
import scipy as sc

sc.__version__

'1.15.1'

In [25]:
import numba

numba.__version__

'0.60.0'

In [26]:
np.__version__

'1.26.4'

In [27]:
# np.show_config()

Great, now let's set up our system and solver with `jitr`:

In [28]:
sys = reactions.ProjectileTargetSystem(
    channel_radius=10 * np.pi,
    lmax=10,
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=40,
    Zproj=0,
)

# COM frame energy
Elab = 14.1

# Lagrange-Mesh R-matrix solver
solver = rmatrix.Solver(40)

# channels holds info for the elastic scattering channel
Elab = 42.1
Ecm, mu, k, eta = utils.kinematics.classical_kinematics(
    sys.mass_target, sys.mass_projectile, Elab, sys.Zproj * sys.Ztarget
)
channels, asymptotics = sys.get_partial_wave_channels(Ecm, mu, k, eta)

In [29]:
# Woods-Saxon potential parameters
V0 = -60  # real potential strength
W0 = -20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness
params = (V0, W0, R0, a0)

In [30]:
fm = solver.free_matrix(sys.channel_radius, sys.l, coupled=False)

In [31]:
b = solver.precompute_boundaries(sys.channel_radius)

In [46]:
# run solver for S-wave
l = 5
R, S, uext_prime_boundary = solver.solve(
    channels[l],
    asymptotics[l],
    woods_saxon_potential,
    params,
    basis_boundary=b,
    free_matrix=fm[l],
)

In [47]:
print(S[0, 0])

(-0.047260622289154945-0.27365461463199625j)


Great, now let's use `scipy` and see if we get the same $\mathcal{S}$-matrix:

In [48]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1

channel_data_rk = reactions.make_channel_data(channels[l])
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol

In [49]:
a = channel_data_rk[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)
print(S_rk)

(-0.047325685095088464-0.27371790878444224j)


In [50]:
100 * (S[0, 0] - S_rk) / S_rk  # percent difference in real and imag parts of S

(-0.026443188018269174+0.019198008006218382j)

Great, our solvers agree up to high precision. Now let's compare the runtime of the two solver options:

In [51]:
%%timeit
R, S, uext_prime_boundary = solver.solve(
    channels[l], asymptotics[l], woods_saxon_potential, params
)

2.5 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%timeit
R, S, uext_prime_boundary = solver.solve(
    channels[l],
    asymptotics[l],
    woods_saxon_potential,
    params,
    basis_boundary=b,
    free_matrix=fm[l],
)

99.9 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Wow, pre-computing the free matrix gives us about a big speedup! What if we precompute the interaction matrix too?

In [53]:
im = solver.interaction_matrix(
    channels[l].k[0],
    channels[l].E[0],
    channels[l].a,
    channels[l].size,
    local_interaction=woods_saxon_potential,
    local_args=params,
)

In [54]:
%%timeit
R, S, uext_prime_boundary = solver.solve(
    channels[l],
    asymptotics[l],
    basis_boundary=b,
    free_matrix=fm[l],
    interaction_matrix=im,
)

54.4 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Another big saving. The point is, depending on your appplication, you may be able to precompute some things and not others. `jitr` gives you the tools to be able to do that. For example, if you have many partial waves to solve for a single set of parameters, and your interaction is $l$-independent, you can precompute the interaction matrix for all partial waves, for that set of parameters. The free matrix, on the other hand, is independent of the parameters, but depends on $l$ (and on energy for multi-channel calculations). This means, for elastic scattering, one can pre-compute the free matrix for each $l$ at the beginning, and reuse for whatever parameter.

In [55]:
%%timeit
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)

41.7 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


On my machine `jitr` is faster by about 150 times, or 500 times if you precompute the free matrix, and like 1000 times if you precompute the interaction matrix as well.

(This does, of course, depend on the solver paramaters; `atol` and `rtol` for `solve_ivp`, and `nbasis` for `LagrangeRMatrixSolver` ).

In [56]:
import pickle

with open("solver.pkl", "wb") as f:
    pickle.dump(solver, f)

In [57]:
with open("solver.pkl", "rb") as f:
    s = pickle.load(f)

In [58]:
%%timeit
R, S, uext_prime_boundary = s.solve(
    channels[l],
    asymptotics[l],
    basis_boundary=b,
    free_matrix=fm[l],
    interaction_matrix=im,
)

54.5 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
